In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [markdown]
# # Libraries

# %% [code]
#Imports
from torch.utils.data import Dataset
import cv2
import openslide
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from tqdm.notebook import tqdm
import time

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# %% [markdown]
# # Load into Pandas

# %% [code]
train = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
train.head()

# %% [markdown]
# # Dataset

# %% [code]
class TrainingDataset(Dataset):
    def __init__ (self, df, labels, transform = None):
        self.df = df
        self.labels = labels
        self.transform = transform
        
    def __len__ (self):
        return len(self.df)
    
    def __getitem__(self, idx):
        slide_identifier = self.df['image_id'].values[idx]
        image_path = '/kaggle/input/prostate-cancer-grade-assessment/train_images/' + slide_identifier + '.tiff'
        slide = openslide.OpenSlide(image_path)
        image = slide.read_region((0,0), slide.level_count - 1, slide.level_dimensions[-1]) #using the smallest image, denoted by [-1] index
        image = cv2.resize(np.asarray(image)[:,:,0:3], (256, 256)) #downsampling to 256x256, slice and remove A index
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            transformed = self.transform(image = image)
            image = transformed['image']
        
        label = self.labels[idx]
        return image, label
    
class TestingDataset(Dataset):
    def __init__ (self, df, folder, transform = None):
        self.df = df
        self.folder = folder
        self.transform = transform
        
    def __len__ (self):
        return len(self.df)
    
    def __getitem__(self, idx):
        slide_identifier = self.df['image_id'].values[idx]
        image_path = '/kaggle/input/prostate-cancer-grade-assessment/' + self.folder + '/'+ slide_identifier + '.tiff'
        slide = openslide.OpenSlide(image_path)
        image = slide.read_region((0,0), slide.level_count - 1, slide.level_dimensions[-1]) #using the smallest image, denoted by [-1] index
        image = cv2.resize(np.asarray(image)[:,:,0:3], (256, 256)) #downsampling to 256x256
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            transformed = self.transform(image = image)
            image = transformed['image']
        

        return image

# %% [markdown]
# # Transformations

# %% [code]
def get_transform(dataset_type):
    assert dataset_type in ('train', 'valid')
    if dataset_type == 'train':

        return Compose([
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    elif dataset_type == 'valid':

        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

# %% [markdown]
# # Split Dataset Into n (4) Folds

# %% [code]
fold_dataset = train.copy()
kfold = StratifiedKFold(n_splits = 4, shuffle = True ,random_state = 42)
gen = kfold.split(fold_dataset, fold_dataset['isup_grade']) #create generator
for fold, (train_,test_) in enumerate(gen): 
    fold_dataset.loc[test_, 'fold'] = fold #assign fold number to indices for test (1/4 of dataset)
fold_dataset['fold'] = fold_dataset['fold'].astype(int) #change to integer
fold_dataset.head()

# %% [markdown]
# # Build Model

# %% [markdown]
# **Resnet34**:
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
# 
# 
# weights: https://download.pytorch.org/models/resnet34-333f7ec4.pth

# %% [code]
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)

# %% [code]
class ResNet34(nn.Module):
    def __init__(self, freeze = True):
        super(ResNet34, self).__init__()
#         layers = []
#         layers.append(nn.Linear(512, 256))
#         layers.append(nn.ReLU())
#         layers.append(nn.Linear(256, 6))


        self.model = resnet34(pretrained=False)
        weights_path = '/kaggle/input/resnet34/resnet34.pth'
        self.model.load_state_dict(torch.load(weights_path))
        self.model.fc = nn.Linear(self.model.fc.in_features, 6)
#         self.model.avg_pool = nn.AdaptiveAvgPool2d(1)
#         self.model.last_linear = nn.Linear(self.model.last_linear.in_features, CFG.target_size)
        
    def forward(self, x):
        x = self.model(x)
        return x

# %% [code]
net = ResNet34()
net

# %% [code]
# class ResNet34(nn.Module):
#     def __init__(self, pretrained, freeze = True):
#         super(ResNet34, self).__init__()
#         if pretrained:
#             self.model = pretrainedmodels.__dict__['resnet34'](pretrained = 'imagenet')
#         else:
#             self.model = pretrainedmodels.__dict__['resnet34'](pretrained = None)
#         if freeze:
#             for param in self.model.parameters():
#                 param.requires_grad = False
#         layers = []
#         layers.append(nn.Linear(512, 256))
#         layers.append(nn.ReLU())
#         layers.append(nn.Linear(256, 6))


#         self.l0 = nn.Sequential(*layers)
#         self.l1 = nn.Sequential(*layers)

#     def forward(self, X):
#         bs, _, _, _ = X.shape
#         X = self.model.features(X)
#         X = nn.functional.adaptive_avg_pool2d(X, 1).reshape(bs, -1)
#         l0 = self.l0(X)
#         l1 = self.l1(X)
#         return l0, l1

# %% [markdown]
# # Train Model

# %% [code]
# ## For Debugging Purposes:
# fold_dataset = fold_dataset.sample(100)

# %% [code]
#  def training_(fold_num, num_epochs):
        
#         # get the dataset from fold_dataset for current fold #################################################################
#         print('Fold number %s:' % fold_num)
        
#         train_df = fold_dataset.loc[fold_dataset['fold'] != fold_num].reset_index(drop = True)
#         valid_df = fold_dataset.loc[fold_dataset['fold'] == fold_num].reset_index(drop = True)
        
#         train_label = train_df['isup_grade']
#         valid_label = valid_df['isup_grade']
        
#         train_dataset = TrainingDataset(train_df, train_label, get_transform(dataset_type = 'train'))
#         valid_dataset = TrainingDataset(valid_df, valid_label, get_transform(dataset_type = 'valid'))
        
#         trainloader = DataLoader(train_dataset, batch_size = 16)
#         validloader = DataLoader(valid_dataset, batch_size = 16)
        
#         # initialize model, and set criterion, optimizer, scheduler #################################################################
# #         net = VGG('VGG16', dropout_rate = 0)
#         net = ResNet34(freeze = False)
#         net.to(device)
#         criterion = nn.CrossEntropyLoss()
#         criterion.to(device)
# #         optimizer = optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)
#         optimizer = optim.Adam(net.parameters(), lr=0.0001, amsgrad=False)
#         scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1, gamma=0.5)
# #         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, eps=1e-6)

        
        
#         lst_train_qwk = np.zeros(num_epochs)
#         lst_valid_qwk = np.zeros(num_epochs)   
        
#         start = time.time()
#         best_qwk = -np.inf
#         for epoch in range(num_epochs):
#             running_loss = 0.0
            
#             print('learning rate: %f' % optimizer.param_groups[0]['lr'])
            
# #             add a get_accuracy function
            
#             net.train()
#             optimizer.zero_grad() 
#             for i, data in enumerate(tqdm(trainloader)):
#                 inputs, labels = data[0].to(device),data[1].to(device)
#                 optimizer.zero_grad() 
#                 outputs = net(inputs)
#                 loss = criterion(outputs, labels)
#                 loss.backward() #backward() is an attribute of the tensor class
#                 optimizer.step()
#                 running_loss += loss.item()



#                 if i % 20 == 19:    # print every 20 mini-batches
#                     print('[%d, %5d] loss: %.3f' %
#                           (epoch + 1, i + 1, running_loss / 20))
#                     running_loss = 0.0
#             running_loss = 0.0
            
#             predictions = []
#             truth_labels = []
#             net.eval()
#             for i, data in enumerate(tqdm(validloader)):
#                 inputs, labels = data[0].to(device),data[1].to(device) 
#                 with torch.no_grad():
#                     outputs_predicted = net(inputs)
#                 loss = criterion(outputs_predicted, labels)
#                 running_loss += loss.item()
                
                
# #                 print(outputs_predicted)
#                 outputs_predicted = outputs_predicted.to('cpu').numpy().argmax(1)
# #                 print(outputs_predicted)
#                 labels = labels.to('cpu').numpy()
                
#                 predictions.append(outputs_predicted)
#                 truth_labels.append(labels)
# #             print(predictions)
# #             print(truth_labels)
#             qwk = cohen_kappa_score(predictions[0], truth_labels[0], weights='quadratic') 
            

#             if qwk > best_qwk:
#                 best_qwk = qwk
#                 print('Saving Best Score ------------->')
#                 PATH = 'foldnum%s'% fold_num + 'ResNet34.pth'
#                 torch.save(net.state_dict(), PATH)
                
#             scheduler.step()
#             print('Current QWK: %s' % qwk)
        
        
#         print('Best QWK: %s' % best_qwk)
#         print('[%d] loss: %.3f' %
#               (epoch + 1, running_loss / 100))


#         print('Finished Fold Training')
#         end = time.time()
#         print('Minutes run for: %s' % ((end - start)/60))
#         return None

# %% [code]
# for fold in range(4):
#     training_(fold,num_epochs = 8)

# %% [markdown]
# # Train Without Cross Validation

# %% [code]
fold_dataset = train.copy()
kfold = StratifiedKFold(n_splits = 10, shuffle = True ,random_state = 42)
gen = kfold.split(fold_dataset, fold_dataset['isup_grade']) #create generator
for fold, (train_,test_) in enumerate(gen): 
    fold_dataset.loc[test_, 'fold'] = fold #assign fold number to indices for test (1/4 of dataset)
fold_dataset['fold'] = fold_dataset['fold'].astype(int) #change to integer
fold_dataset.head()

# %% [code]
 def training_(fold_num, num_epochs):
        
        # get the dataset from fold_dataset for current fold #################################################################
        print('Fold number %s:' % fold_num)
        
        train_df = fold_dataset.loc[fold_dataset['fold'] != fold_num].reset_index(drop = True)
        valid_df = fold_dataset.loc[fold_dataset['fold'] == fold_num].reset_index(drop = True)
        
        train_label = train_df['isup_grade']
        valid_label = valid_df['isup_grade']
        
        train_dataset = TrainingDataset(train_df, train_label, get_transform(dataset_type = 'train'))
        valid_dataset = TrainingDataset(valid_df, valid_label, get_transform(dataset_type = 'valid'))
        
        trainloader = DataLoader(train_dataset, batch_size = 16)
        validloader = DataLoader(valid_dataset, batch_size = 16)
        
        # initialize model, and set criterion, optimizer, scheduler #################################################################
#         net = VGG('VGG16', dropout_rate = 0)
        net = ResNet34(freeze = False)
        net.to(device)
        criterion = nn.CrossEntropyLoss()
        criterion.to(device)
#         optimizer = optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)
        optimizer = optim.Adam(net.parameters(), lr=0.0001, amsgrad=False)
#         scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1, gamma=0.5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, eps=1e-6)

        
        
        lst_train_qwk = np.zeros(num_epochs)
        lst_valid_qwk = np.zeros(num_epochs)   
        
        start = time.time()
        best_qwk = -np.inf
        for epoch in range(num_epochs):
            running_loss = 0.0
            
            print('learning rate: %f' % optimizer.param_groups[0]['lr'])
            
#             add a get_accuracy function
            
            net.train()
            optimizer.zero_grad() 
            for i, data in enumerate(tqdm(trainloader)):
                inputs, labels = data[0].to(device),data[1].to(device)
                optimizer.zero_grad() 
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward() #backward() is an attribute of the tensor class
                optimizer.step()
                running_loss += loss.item()



                if i % 20 == 19:    # print every 20 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 20))
                    running_loss = 0.0
            
            running_loss = 0.0
            predictions = []
            truth_labels = []
            net.eval()
            for i, data in enumerate(tqdm(validloader)):
                inputs, labels = data[0].to(device),data[1].to(device) 
                with torch.no_grad():
                    outputs_predicted = net(inputs)
                loss = criterion(outputs_predicted, labels)
                running_loss += loss.item()
                
                
#                 print(outputs_predicted)
                outputs_predicted = outputs_predicted.to('cpu').numpy().argmax(1)
#                 print(outputs_predicted)
                labels = labels.to('cpu').numpy()
                
                predictions.append(outputs_predicted)
                truth_labels.append(labels)
#             print(predictions)
#             print(truth_labels)
            qwk = cohen_kappa_score(predictions[0], truth_labels[0], weights='quadratic') 
            

            if qwk > best_qwk:
                best_qwk = qwk
                print('Saving Best Score ------------->')
                PATH = 'foldnum%s'% fold_num + 'ResNet34.pth'
                torch.save(net.state_dict(), PATH)
                
            scheduler.step(running_loss)
            print('Current QWK: %s' % qwk)
        
        
        print('Best QWK: %s' % best_qwk)
        print('[%d] loss: %.3f' %
              (epoch + 1, running_loss / 100))


        print('Finished Fold Training')
        end = time.time()
        print('Minutes run for: %s' % ((end - start)/60))
        return None

# %% [code]
fold = 7 #pick first fold
training_(fold,num_epochs = 10)

# %% [markdown]
# # Inference and Submission

# %% [code]
# def predict_test(net, dataloader):
#     predicted_mat = []
#     net.to(device)
#     for i, images in enumerate(tqdm(dataloader)):
#         images = images.to(device)
#         with torch.no_grad():
#             predicted = net(images)
#         predicted = predicted[0].to('cpu')
#         predicted = predicted.numpy()
#         predicted_mat.append(predicted)
#     return predicted_mat

# %% [code]
# def submit(test_df, image_folder = 'test_images'):
#     if os.path.exists('/kaggle/input/prostate-cancer-grade-assessment/' + image_folder):
#         print('Running inference ---------->')
#         test_dataset = TestingDataset(test_df, image_folder, get_transform(dataset_type = 'valid'))
#         testloader = DataLoader(test_dataset, batch_size = 16, shuffle = False)
#         probs = []
#         for fold_num in range(4):
#             PATH = 'foldnum%s'% fold_num + 'ResNet34.pth'
#             net = ResNet34(pretrained = False, freeze = False)
#             net.load_state_dict(torch.load(PATH, map_location = device)) #might need to be changed
#             predicted = predict_test(net, testloader)
#             probs.append(predicted)
#         probs_avg = np.mean(probs,0)
#         predictions = probs_avg[0].transpose().argmax(0)
#         test_df['isup_grade'] = predictions
# #         test_df.insert(len(test_df.columns), 'isup_grade', predictions, allow_duplicates=True) #adds to last column
#     else:
#         print('directory not found')
#     return test_df

# %% [code]
# sampledf = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv')
# submission = submit(sampledf, image_folder = 'test_images')
# submission['isup_grade'] = submission['isup_grade'].astype(int)
# submission.to_csv('submission.csv', index=False)
# submission.head()

# %% [code]


# %% [code]
